# Tải các thư viện cần thiết

In [1]:
!pip -q install transformers torch accelerate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.8 MB/s eta 0:00:00


# Import các thư viện

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
import pandas as pd
import random

# Tải file dataset từ Google Drive

In [3]:
!gdown 152eRXERoaQE02K6yAnA7EvouVRMEp-bH

Downloading...
From: https://drive.google.com/uc?id=152eRXERoaQE02K6yAnA7EvouVRMEp-bH
To: /content/train_processed.csv
100% 36.1M/36.1M [00:00<00:00, 77.9MB/s]


In [4]:
df = pd.read_csv("/content/train_processed.csv")
df.head(3)

,score,word_count,essay_length,sentences_count,paragraph_count,clean_text,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,misspelled_count
0,3,498,2677,13,1,many people have car where they live. the thin...,56.97,15.1,11.4,7.21,18.0,8.40,57.0,13.000000,26
1,3,332,1669,19,9,i am a scientist at nasa that is discussing th...,87.35,5.5,6.8,4.52,5.9,6.11,22.0,6.714286,8
2,4,550,3077,24,7,people always wish they had the same technolog...,65.05,9.9,11.4,8.36,11.1,6.92,59.0,15.500000,9


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   score                         17307 non-null  int64  
 1   word_count                    17307 non-null  int64  
 2   essay_length                  17307 non-null  int64  
 3   sentences_count               17307 non-null  int64  
 4   paragraph_count               17307 non-null  int64  
 5   clean_text                    17307 non-null  object 
 6   flesch_reading_ease           17307 non-null  float64
 7   flesch_kincaid_grade          17307 non-null  float64
 8   smog_index                    17307 non-null  float64
 9   coleman_liau_index            17307 non-null  float64
 10  automated_readability_index   17307 non-null  float64
 11  dale_chall_readability_score  17307 non-null  float64
 12  difficult_words               17307 non-null  float64
 13  l

# Cài đặt và cấu hình model

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# cấu hình cho model
config = AutoConfig.from_pretrained("google/flan-t5-base", trust_remote_code = True)
config.init_device = "cuda"
config.temperature = 0.1
config.max_length = 300
config.eos_token_id = tokenizer.eos_token_id
config.pad_token_id = tokenizer.pad_token_id
config.do_sample = True

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map = "cuda", config = config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Chọn một `essay` ngẫu nhiên từ cột `clean_text`

In [7]:
# Choose a random 'clean_text' row from the dataframe as the input text
input_text = df['clean_text'][random.randint(0, len(df))]
print("Input text:", input_text)

Input text: in this article they are talking about a way calculate our emothins called facs. prof. thomas huang is the creater of this technology with of course some other people. it is a highly advanced mood ring of sourts but intead of just guessing, it make a 3d model of you face that allows it to calculate what you feeling. thomas huang paired with an psycholgists to determin the different sign of emotion so he would be able to prepare the machine. this idea could change the way we learn today; we realy on our technology so much today kid now day do the bare minimum to pas. most of the pappers we do we are googling the awnsers. it is about time we find a way to teach the future gerenation. if we had this kind of technology scanning u and being able to tell when the kid do not uderstand what they are being taught it could possibly boost our learning and help better u for the future. i know from expersience that if i am not aware of what i am being taught i will just look up the awns

# Chức năng 1: Tóm tắt văn bản

In [8]:
summary_command = 'Summary this text: {}'.format(input_text)
print('-> Command:', summary_command)

input_ids = tokenizer(summary_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('\n->Answer:\n', tokenizer.decode(outputs[0]))

-> Command: Summary this text: in this article they are talking about a way calculate our emothins called facs. prof. thomas huang is the creater of this technology with of course some other people. it is a highly advanced mood ring of sourts but intead of just guessing, it make a 3d model of you face that allows it to calculate what you feeling. thomas huang paired with an psycholgists to determin the different sign of emotion so he would be able to prepare the machine. this idea could change the way we learn today; we realy on our technology so much today kid now day do the bare minimum to pas. most of the pappers we do we are googling the awnsers. it is about time we find a way to teach the future gerenation. if we had this kind of technology scanning u and being able to tell when the kid do not uderstand what they are being taught it could possibly boost our learning and help better u for the future. i know from expersience that if i am not aware of what i am being taught i will ju

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



->Answer:
 <pad> awnsers are a way to calculate our emotions</s>


# Chức năng 2: Chấm điểm văn bản

In [9]:
score_command = 'Try score this text on the scale of 1 to 6: {}'.format(input_text)
print('Command:', score_command)

input_ids = tokenizer(score_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('Answer:\n', tokenizer.decode(outputs[0]))

Command: Try score this text on the scale of 1 to 6: in this article they are talking about a way calculate our emothins called facs. prof. thomas huang is the creater of this technology with of course some other people. it is a highly advanced mood ring of sourts but intead of just guessing, it make a 3d model of you face that allows it to calculate what you feeling. thomas huang paired with an psycholgists to determin the different sign of emotion so he would be able to prepare the machine. this idea could change the way we learn today; we realy on our technology so much today kid now day do the bare minimum to pas. most of the pappers we do we are googling the awnsers. it is about time we find a way to teach the future gerenation. if we had this kind of technology scanning u and being able to tell when the kid do not uderstand what they are being taught it could possibly boost our learning and help better u for the future. i know from expersience that if i am not aware of what i am 

# Chức năng 3: Dịch thuật văn bản

In [10]:
translate_command = 'Translate this text into French: {}'.format(input_text)
print('Command:', translate_command)

input_ids = tokenizer(translate_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('Answer:\n', tokenizer.decode(outputs[0]))

Command: Translate this text into French: in this article they are talking about a way calculate our emothins called facs. prof. thomas huang is the creater of this technology with of course some other people. it is a highly advanced mood ring of sourts but intead of just guessing, it make a 3d model of you face that allows it to calculate what you feeling. thomas huang paired with an psycholgists to determin the different sign of emotion so he would be able to prepare the machine. this idea could change the way we learn today; we realy on our technology so much today kid now day do the bare minimum to pas. most of the pappers we do we are googling the awnsers. it is about time we find a way to teach the future gerenation. if we had this kind of technology scanning u and being able to tell when the kid do not uderstand what they are being taught it could possibly boost our learning and help better u for the future. i know from expersience that if i am not aware of what i am being taugh

# Sẽ cải thiện thêm tương lai